Dans cette section, nous réentrainons notre modèle avec les données de user3. Le processus reste le même que celui réalisé avec le dataset 
users1.


---



---



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
df_user_data = pd.read_excel("C:/Users/asus/Desktop/Datasets Cooding-Week/Livrable_code_g_colab/datasets_excel/user3_base.xlsx")
df_user_data_test = pd.read_excel("C:/Users/asus/Desktop/Datasets Cooding-Week/Livrable_code_g_colab/datasets_excel/user3_test.xlsx")

In [4]:
headers = ["userid", "itemid", "rating", "timestamp"]
df_user_data.columns = headers
df_user_data_test.columns = headers

In [6]:
R = np.zeros((943, 1682))
zeros =np.zeros((943, 1))

for i in range(0, 80000):
    n = df_user_data["userid"][i]
    m = df_user_data["itemid"][i]
    c = df_user_data["rating"][i]
    R[n-1][m-1]= c

**MODELE 1: SIMPLE OVERALL AVERAGE MODEL**

In [7]:
x_counts = 0
total = 0
for j in range(0, 1682):
    for i in range(0, 943):
        total += R[i][j]
        if R[i][j] ==0 :
            x_counts+=1

moyenne_mu = total / (943*1682 - x_counts)
moyenne_mu

3.5310875

In [8]:
df_user_data_test['mean_rating'] = moyenne_mu
df_user_data_test.head()

,userid,itemid,rating,timestamp,mean_rating
0,1,5,3,889751712,3.531087
1,1,11,2,875072262,3.531087
2,1,16,5,878543541,3.531087
3,1,25,4,875071805,3.531087
4,1,35,1,878542420,3.531087


In [9]:
rmse_1 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating'])
rmse_1

1.2356151664062502

**MODELE 2 : MOVIE EFFECT**

In [10]:
# CREATION D'UNE LISTE CONTENANT LES BIAIS DE CHAQUE FILMS EN PRNANT EN COMPTE LES FILMS N'AYANT PAS ÉTÉ NOTÉS

bias = []

for i in range(0, 1682):
    
    somme = 0
    count = 0
    for j in range(0, 943):
        if R[j][i] != 0:
            count += 1
            somme  += R[j][i]
    if count != 0:
        b = (somme / count) - moyenne_mu
        bias.append(b)

    else:
        b = 0
        bias.append(b)

In [11]:
predicted_rating_2 = list(bias + moyenne_mu )

df_user_data_test['mean_rating_2'] = 0

In [12]:
#Definition et completion de mean_rating_2 dans le dataset
for i in range(len(df_user_data_test["itemid"])):
    df_user_data_test["mean_rating_2"][i]= predicted_rating_2[df_user_data_test["itemid"][i]-1]

In [13]:
# CALCUL DU ROOT MEAN SQUARE ERROR 
rmse_2 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating_2'])
rmse_2

1.0397125919944021

**MODELE 3 : MOVIE AND USERS EFFECT**

In [14]:
# CREATION D'UNE LISTE CONTENANT LES BIAIS DE CHAQUE UTILISATEUR
bias_u = []

for j in range(0, 943):
    
    somme = 0
    count = 0
    for i in range(0, 1642):
        if R[j][i] != 0:
            count += 1
            somme  += R[j][i] - moyenne_mu - bias[i]
    if count != 0:
        b_u = (somme / count)
        bias_u.append(b_u)

In [15]:
R_predicted_rating_3 = np.zeros((943, 1682)) # ICI, nous avons maintenant deux facteurs a prendre en compte (le biais relatif a chacun des films et celui de chaque utilisateurs), ce 
                                              # qui justifie la creation d'un tableau a deux dimensions

In [16]:
for i in range(0, 943):
    for j in range(0, 1482):
        R_predicted_rating_3[i][j]= moyenne_mu + bias[j] + bias_u[i]

In [17]:
df_user_data_test['mean_rating_3'] = 0

for i in range(len(df_user_data_test["itemid"])):
    df_user_data_test["mean_rating_3"][i]= R_predicted_rating_3[df_user_data_test["userid"][i]-1][df_user_data_test["itemid"][i]-1]

In [18]:
rmse_3 = sklearn.metrics.mean_squared_error(df_user_data_test['rating'], df_user_data_test['mean_rating_3'])
rmse_3

0.9430971366624057

**MODELE 4 : REGULARIZED MOVIE AND USER EFFECT**

In [19]:
rmses = []

lambdas = np.arange(1.0, 1.50, 0.10)

R_predicted_rating_4 = np.zeros((943, 1682))

In [20]:
# Que represente l
for l in lambdas:
    mu = np.mean(df_user_data['rating'])

    #b_i = df_user_data.groupby('itemid').agg(b_i=('rating', lambda x: np.sum(x - mu) / (len(x) + l)))

    b_i = []
    # Definition de b_i
    for i in range(0, 1682):
        
        somme = 0
        count = 0
        for j in range(0, 943):
            if R[j][i] != 0:
                count += 1
                somme  += R[j][i]
        if count != 0:
            b = (somme - moyenne_mu*count )/ (count + l)
            b_i.append(b)

        else:
            b = 0
            b_i.append(b)

    b_u2 = []

    for j in range(0, 943):
        
        somme = 0
        count = 0
        for i in range(0, 1642):
            if R[j][i] != 0:
                count += 1
                somme  += (R[j][i] - moyenne_mu - bias[i])
        if count != 0:
            b = (somme / count)
            b_u2.append(b)


    for i in range(0, 943):
        for j in range(0, 1482):
            R_predicted_rating_4[i][j]= moyenne_mu + b_i[j] + b_u2[i]
    #print(R_predicted_rating_3)

    df_user_data_test['mean_rating_4'] = 0

    for i in range(len(df_user_data_test["itemid"])):
        df_user_data_test["mean_rating_4"][i]= R_predicted_rating_4[df_user_data_test["userid"][i]-1][df_user_data_test["itemid"][i]-1]

    

    rmses.append(sklearn.metrics.mean_squared_error(df_user_data_test["mean_rating_4"], df_user_data_test['rating']))
    df_user_data_test['b_i'] = 0 
    df_user_data_test['b_u'] = 0 


    #Definition et completion de mean_rating_2 dans le dataset
    for i in range(len(df_user_data_test["itemid"])):
        df_user_data_test["b_i"][i]= b_i[df_user_data_test["itemid"][i]-1]
        df_user_data_test["b_u"][i]= b_u2[df_user_data_test["userid"][i]-1]

In [21]:
rmses

[0.9361309451031049,
 0.9358559829856967,
 0.9356137451863425,
 0.9353999841957038,
 0.9352112010961049]